<a href="https://colab.research.google.com/github/dfrepos/biocomp/blob/main/biocompFotonica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Modelagem de Rede Neural Artificial para substituição de simulação de dispositivos fotônicos.**
Este notebook carrega a base de dados simulados numericamente, que estão hospedados no github. 
Para mais informações sobre como foram gerados os indivíduos aleatórios, acessar [este](https://docs.google.com/document/d/1oPG2PWgAsiPtcvPfIa2L_S_nJEU16c26hBEiFvPiMo4/edit?usp=sharing) documento.

# Importando Bibliotecas

In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Activation,Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy



# Importando base de dados
Os dados são importados para um DataFrame com a seguinte estrutura

[Lower1550, Lower2000, Upper1550, Upper2000, elemento_1, elemento_2, ..., elemento_600]

In [3]:
df = pd.read_csv('https://github.com/dfrepos/biocomp/blob/main/results280121.csv?raw=true') #dfrprojects


In [4]:
df.shape

(10000, 604)

Reduzindo em 90% o tamanho do dataset para testar o modelo

In [9]:
df = df.sample(frac=0.1, random_state=1337)

Separando o dataset em set de treino e de validação

In [11]:
val_df = df.sample(frac=0.2, random_state=1337)
train_df = df.drop(val_df.index)

print(
    "Using %d samples for training and %d for validation"
    % (len(train_df), len(val_df))
)

Using 160 samples for training and 40 for validation


# Rede Neural Artificial (em desenvolvimento)

Fazer:
- Entender qual o formato da entrada RNA e adaptar do formato do DataFrame (variável df)
- Definir arquitetura das camadas ocultas (a entrada tem 600 e a saída 4)

## Definindo os datasets de treinamento e de teste

In [61]:
X_train = df.iloc[0:500,4:].values.tolist()
Y_train = df.iloc[0:500,0:4].values.tolist()
X_test = df.iloc[500:1500,4:].values.tolist()
Y_test = df.iloc[500:1500,4:].values.tolist()

In [14]:
def df_to_dataset(df):
    df = df.copy()
    labels = df.pop("Lower1550")
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    ds = ds.shuffle(buffer_size=len(df))
    return ds
    
train_ds = df_to_dataset(train_df)
val_ds = df_to_dataset(val_df)

In [15]:
for x, y in train_ds.take(1):
    print("Input:", x)
    print("Target:", y)

Input: {'Lower2000': <tf.Tensor: shape=(), dtype=float64, numpy=0.0367557771933895>, 'Upper1550': <tf.Tensor: shape=(), dtype=float64, numpy=0.0218241828073136>, 'Upper2000': <tf.Tensor: shape=(), dtype=float64, numpy=0.0238984507484357>, 'elemento_1': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'elemento_2': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'elemento_3': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'elemento_4': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'elemento_5': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'elemento_6': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'elemento_7': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'elemento_8': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'elemento_9': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'elemento_10': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'elemento_11': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'elemento_12': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'elemento_13': <tf.Tensor: shape=(), dtype=int64, 

In [16]:
train_ds = train_ds.batch(32)
val_ds = val_ds.batch(32)

## Definindo o modelo RNA

A camada de entrada deve ter 600 neurônios, a de saída apenas 4.

A ideia é que a última camada tenha ativação ReLU, que oferece saída 'analógica'.

In [18]:
model = Sequential([ 
  Dense(units = 50, input_shape = (600,), activation = 'relu', input_dim = 600),
  Dense(units = 25, activation = 'relu'),
  Dense(units = 10, activation = 'relu'),
  Dense(units = 4, activation = 'relu')
])

#  Dense(units = 50, input_shape = (600,), activation = 'relu', input_dim = 600),

In [19]:
model.summary() # Essa expressão permite visualizar o modelo antes de executá-lo

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                30050     
_________________________________________________________________
dense_1 (Dense)              (None, 25)                1275      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                260       
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 44        
Total params: 31,629
Trainable params: 31,629
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [23]:
model.fit(x=train_ds, batch_size=10, epochs=30, shuffle=True, verbose=2)

Epoch 1/30


ValueError: ignored